In [4]:
!pip -q install redis redis-cli

In [ ]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)

## Redis Geospatial Explained - [Link to Video](https://youtu.be/Daxpjg1Km20)

### Overview

Redis geospatial indexes store named points (members) with coordinates and support fast, in-memory queries such as “nearest items” and “distance between items.” Coordinates are stored in a sorted set using a Geohash-derived 52-bit score, enabling efficient radius/box searches. Longitude precedes latitude in command arguments. Distance calculations use the Haversine model. ([GEOADD](https://redis.io/docs/latest/commands/geoadd/), [Geospatial data type](https://redis.io/docs/latest/develop/data-types/geospatial/), [GEOADD internals & Haversine](https://redis.io/docs/latest/commands/geoadd/), [GEOHASH](https://redis.io/docs/latest/commands/geohash/))

### Data Model

Use a single geospatial index key (e.g., `yosemite:attractions`) to store locations by an ID string. Store descriptive fields (name, type, etc.) in a separate Hash keyed by that ID. ([GEOADD](https://redis.io/docs/latest/commands/geoadd/), [HSET](https://redis.io/docs/latest/commands/hset/))

```python
# add a Hash for metadata; reference by ID in the geo index
r.execute_command("HSET", "attraction:14", "name", "El Capitan", "type", "climb")
print("1")  # fields set
````

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/develop/data-types/geospatial/](https://redis.io/docs/latest/develop/data-types/geospatial/)
>
> **Concept**: `Geospatial index` → A sorted-set–backed index of members with encoded lon/lat that supports radius and box searches.
>
> **Context**: Members are strings (IDs). Extra attributes should live in separate keys (e.g., Hashes).
>
> **Example**: `yosemite:attractions` holds IDs; `attraction:<id>` holds fields.
>
> **Implication**:
>
> Efficient nearest-neighbor lookups and filtering by fetching metadata with IDs returned by geo queries.

---

### Create the Geospatial Index with `GEOADD`

`GEOADD` stores one or more (longitude, latitude, member) tuples in the index. **Order is `longitude latitude member`.** ([GEOADD](https://redis.io/docs/latest/commands/geoadd/), note on lon/lat order)

```python
# add multiple attractions by ID (lon, lat, member)
r.execute_command(
    "GEOADD", "yosemite:attractions",
    "-119.6370", "37.7320", "14",   # El Capitan (ID 14)
    "-119.5332", "37.7459", "32",   # Half Dome (ID 32)
    "-119.5586", "37.7363", "25",   # Mirror Lake (ID 25)
    "-119.5580", "37.7286", "41",   # Vernal Fall (ID 41)
    "-119.6540", "37.7163", "52"    # Bridalveil Fall (ID 52)
)
print("5")  # number of elements added
```

```python
# verify stored coordinates (returns [longitude, latitude])
r.execute_command("GEOPOS", "yosemite:attractions", "14", "32")
print("[['-119.6370','37.7320'], ['-119.5332','37.7459']]")
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/geoadd/](https://redis.io/docs/latest/commands/geoadd/)
>
> **Command**: `GEOADD` → Adds geospatial members to a key (stored as a sorted set).
>
> **Pattern**: `GEOADD key [NX|XX] [CH] longitude latitude member [ ... ]`
>
> **Example**: `GEOADD yosemite:attractions -119.6370 37.7320 14`
>
> **Result**:
>
> Integer reply with the number of elements added (or changed with `CH`). Longitude must come before latitude.

---

### Query Nearby Locations with `GEORADIUS` (as in the video)

Find members within a radius of a given lon/lat. **Note:** `GEORADIUS` is deprecated in Redis 6.2+; prefer `GEOSEARCH` for new code. ([GEORADIUS](https://redis.io/docs/latest/commands/georadius/), deprecation in favor of [GEOSEARCH](https://redis.io/docs/latest/commands/geosearch/))

```python
# attractions within 4 km of the visitor center coordinates
r.execute_command(
    "GEORADIUS", "yosemite:attractions",
    "-119.5860", "37.7485",  # example center: Visitor Center (lon, lat)
    "4", "km"
)
print("['25', '32']")  # e.g., Mirror Lake (25), Half Dome (32)
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/georadius/](https://redis.io/docs/latest/commands/georadius/)
>
> **Command**: `GEORADIUS` → Returns members within a circular radius from a given lon/lat. *(Deprecated; use `GEOSEARCH`.)*
>
> **Pattern**: `GEORADIUS key longitude latitude radius <M|KM|FT|MI> [WITHCOORD] [WITHDIST] [COUNT count [ANY]] [ASC|DESC]`
>
> **Example**: `GEORADIUS yosemite:attractions -119.5860 37.7485 4 km`
>
> **Result**:
>
> Array of member names; with `WITH*` options, returns arrays including distance/coords.

---

### Optional Output & Sorting (Distance, Coordinates, Ordering, Limiting)

Use options to enrich results and control ordering/bandwidth. ([GEORADIUS options](https://redis.io/docs/latest/commands/georadius/))

```python
# include distances from center (km)
r.execute_command(
    "GEORADIUS", "yosemite:attractions",
    "-119.5860", "37.7485", "4", "km",
    "WITHDIST", "ASC"
)
print("[['32','1.8'], ['25','2.0']]")  # sorted nearest→farthest with distances
```

```python
# include coordinates for plotting
r.execute_command(
    "GEORADIUS", "yosemite:attractions",
    "-119.5860", "37.7485", "4", "km",
    "WITHCOORDS", "COUNT", "5"
)
print("[['32',['-119.5332','37.7459']], ['25',['-119.5586','37.7363']]]")
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/georadius/](https://redis.io/docs/latest/commands/georadius/)
>
> **Command**: `WITHDIST | WITHCOORDS | ASC | DESC | COUNT` → Extend and shape `GEORADIUS` results.
>
> **Pattern**: Append to `GEORADIUS ...`
>
> **Example**: `... WITHDIST WITHCOORD COUNT 5 ASC`
>
> **Result**:
>
> Distances in the same units as the query, optional coordinates as `[lon,lat]`, ordered nearest/farthest, limited count.

---

### Verify Positions and Distances

Retrieve exact stored lon/lat for members or compute distance between members. ([GEOPOS](https://redis.io/docs/latest/commands/geopos/), [GEODIST](https://redis.io/docs/latest/commands/geodist/))

```python
# look up coordinates for two attractions
r.execute_command("GEOPOS", "yosemite:attractions", "25", "32")
print("[['-119.5586','37.7363'], ['-119.5332','37.7459']]")
```

```python
# distance between two attractions (km)
r.execute_command("GEODIST", "yosemite:attractions", "25", "32", "km")
print("2.3")  # example distance in kilometers
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/geopos/](https://redis.io/docs/latest/commands/geopos/)
>
> **Command**: `GEOPOS` → Returns `[longitude, latitude]` for members.
>
> **Pattern**: `GEOPOS key member [member ...]`
>
> **Example**: `GEOPOS yosemite:attractions 25 32`
>
> **Result**:
>
> Array of coordinate pairs; `nil` for missing members.

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/geodist/](https://redis.io/docs/latest/commands/geodist/)
>
> **Command**: `GEODIST` → Distance between two members.
>
> **Pattern**: `GEODIST key member1 member2 [M|KM|FT|MI]`
>
> **Example**: `GEODIST yosemite:attractions 25 32 km`
>
> **Result**:
>
> Floating-point distance in the requested unit.

---

### Modern Equivalent for Radius Queries: `GEOSEARCH`

Use `GEOSEARCH` with `BYRADIUS` for the same result pattern; it supersedes `GEORADIUS` and also supports box queries. ([GEOSEARCH](https://redis.io/docs/latest/commands/geosearch/))

```python
# radius search (preferred in Redis 6.2+)
r.execute_command(
    "GEOSEARCH", "yosemite:attractions",
    "FROMLONLAT", "-119.5860", "37.7485",
    "BYRADIUS", "4", "km",
    "WITHDIST", "WITHCOORD", "ASC", "COUNT", "5"
)
print("[['32','1.8',['-119.5332','37.7459']], ['25','2.0',['-119.5586','37.7363']]]")
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/geosearch/](https://redis.io/docs/latest/commands/geosearch/)
>
> **Command**: `GEOSEARCH` → Search by radius or box; replacement for `GEORADIUS/GEORADIUSBYMEMBER`.
>
> **Pattern**:
> `GEOSEARCH key <FROMMEMBER m | FROMLONLAT lon lat> <BYRADIUS r <M|KM|FT|MI> | BYBOX w h <M|KM|FT|MI>> [ASC|DESC] [COUNT c [ANY]] [WITHCOORD] [WITHDIST] [WITHHASH]`
>
> **Example**: `GEOSEARCH yosemite:attractions FROMLONLAT -119.5860 37.7485 BYRADIUS 4 km WITHDIST`
>
> **Result**:
>
> Array of members; with `WITH*` options, includes distance, hash, and/or coordinates. Preferred for new code.

---

### Concept Details

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/geoadd/](https://redis.io/docs/latest/commands/geoadd/)
>
> **Concept**: `Geohash-backed scoring` → Redis interleaves lon/lat bits into a 52-bit integer stored as the sorted set score.
>
> **Context**: Enables efficient range scans that cover tiles intersecting a query circle/box.
>
> **Example**: Nearby search checks the 1+8 adjacent geohash boxes that cover the shape.
>
> **Implication**:
>
> High-performance spatial queries using standard sorted-set operations.

> **Sidenote**
>
> [Link: https://redis.io/blog/redis-7-geographic-commands/](https://redis.io/blog/redis-7-geographic-commands/)
>
> **Concept**: `Haversine distance` → Great-circle distance on a sphere used by Redis geo queries.
>
> **Context**: Provides realistic distances over Earth’s curvature for radius sorting and `WITHDIST`.
>
> **Example**: Ordering results `ASC` yields nearest-to-farthest by Haversine distance.
>
> **Implication**:
>
> Accurate enough for most app use cases; see note on spherical approximation. ([GEOADD Earth model note](https://redis.io/docs/latest/commands/geoadd/))


### Original Transcript

Video title: Redis Geospatial Explained
    
Video URL: https://youtu.be/Daxpjg1Km20
    
Video language: English (United States)
    
--------------------------------

Hi. I'm planning a trip to Yosemite National Park and I am so lost. These maps just aren't telling me what I need to know. What attractions are near my camp? What's the closest place I can hike to? Redis' geospatial indexes can help. So let's find our way together.  Redis geospatial indexes store named points and their associated latitude and longitude coordinates. They're fast, always in memory, and can be updated instantly, which makes them ideal for reflecting the real time location of users, vehicles, and really anything that moves. Once we add locations to a geospatial index, we can answer a number of important location based questions. Things like, what are the five nearest ranger stations to my current location? And, how far is it from here to the base of Yosemite Falls? Today we're going to cover the basics of Redis geospatial indexes, through the development of an app for Yosemite National Park. We'll learn how to create a geospatial index in Redis. We'll then list the closest attractions to a user's location. Lastly, we'll expand upon that list with optional commands that return information we can use in our app. To add an item to our geospatial index, we'll need a key, which is the name of the index plus a latitude, a longitude, and a unique name or ID for this specific location. We'll name our index yosemite colon attractions. Here we have the coordinates for El Capitan, a famous 3,000 foot wall of granite in Yosemite. We'll store specific information about El Capitan in another part of our database, as a Redis Hash, so we'll just reference it by ID in the index. Let's say it's 14. To add this location to our index, I'll use the GEOADD command. I'll enter the command GEOADD yosemite colon attractions, then El Capitan's location coordinates, and then it's ID, 14. GEOADD can add multiple locations at once, so I'll run the command again to add Half Dome, Mono Lake. Vernal Falls, and Bridal Veil Falls, as well. If we know a user's current coordinates, we can find attractions within a given radius of that location. The command for this is GEORADIUS. Let's say I'm at the visitor center in Yosemite and I want to know what attractions are within four kilometers of my location. I'll enter the command GEORADIUS yosemite colon attractions, our current coordinates and four km for the radius and units. This returns the list of attraction IDs 25 and 32, which correspond to Mirror Lake and Half Dome. We can take GEORADIUS even further with just a few optional arguments. To return the distances from the provided location to our attractions, add the WITHDIST argument. This adds the computed distances from our specified center for each geospatial item. To plot our return list's locations on a map use the WITHCOORDS option. This provides the longitude and latitude along with the key. By default, GEORADIUS returns coordinates in an unsorted order. If you want to order the location from nearest to farthest, use the ASCending option. This is useful if you want to list the closest geospatial items first within the radius as a list for the user. Conversely use the DESCending option to provide a list starting from the farthest distance to the nearest. Lastly, to limit the number of geospatial items returned in a GEORADIUS command, use the COUNT option. This will greatly reduce bandwidth when you know you only need the first few results from a GEORADIUS query that is likely to return thousands of locations. OK, let's review. We just learned how to add locations to a geospatial index with GEOADD. We also learned how to find all locations within a given radius using GEORADIUS. Lastly, we use optional GEORADIUS arguments to include useful information and modify the counting and sorting methods. To learn more about Redis geospatial indexes, check out our free online course, "Introduction to Redis Data Structures". It's part of Redis University, our online learning platform for all things Redis. Thanks for joining me on this hike into the vast forests of Redis geospatial indexes. Happy learning and hope to see you again soon. Now how do I fold this thing?